# What are the top radicals and saturates?

- inputs: model, ckcsv

- outputs: radicals and saturates in top concentrations

In [1]:
import os
from rmgpy.chemkin import loadChemkinFile, getSpeciesIdentifier
from pychemkin.chemkin import getMoleFraction
from utilities.wikiMagics import JinjaMagics
from IPython.core.magic import register_cell_magic
ip = get_ipython()
ip.register_magics(JinjaMagics)

In [2]:
# load chemkin model
mech = 'pdd_scratch_add11'
path = os.path.abspath('./')
chemkinPath= path + '/data/' + mech + '/chem.inp'
dictionaryPath = path + '/data/' + mech + '/species_dictionary.txt'
species_list, reactions_list = loadChemkinFile(chemkinPath, dictionaryPath, readComments = False)

:root:Thermo file has default temperature range 300.0 to 1000.0 and 1000.0 to 5000.0


Thermo file has default temperature range 300.0 to 1000.0 and 1000.0 to 5000.0


In [3]:
import re
from rmgpy.molecule.draw import MoleculeDrawer

mech_path = path + '/data/' + mech
if not os.path.isdir(os.path.join(mech_path,'species')):
    os.makedirs(os.path.join(mech_path,'species'))
species = species_list[:]
re_index_search = re.compile(r'\((\d+)\)$').search
for spec in species:
    match = re_index_search(spec.label)
    if match:
        spec.index = int(match.group(0)[1:-1])
        spec.label = spec.label[0:match.start()]
    # Draw molecules if necessary
    fstr = os.path.join(mech_path, 'species', '{0}.png'.format(spec))
    if not os.path.exists(fstr):
        try:
            MoleculeDrawer().draw(spec.molecule[0], 'png', fstr)
        except IndexError:
            raise OutputError("{0} species could not be drawn!".format(getSpeciesIdentifier(spec)))

In [4]:
# load ckcsv info
species_id_list = []
for spc in species_list:
    species_id_list.append(getSpeciesIdentifier(spc))

ckcsvPath= path + '/data/' + mech + '/CKSoln.ckcsv'
tempData, spcData = getMoleFraction(ckcsvPath, species_id_list)

In [5]:
# get top radicals and saturates list
spc_sorted = sorted(spcData.items(), key=lambda tup: -tup[1][0][-1]) # tup:(spc,[array,..])

top_rad_list = []
top_sat_list = []
for tup in spc_sorted:
    spc_id = tup[0]
    final_mf = tup[1][0][-1]
    for spc in species_list:
        if getSpeciesIdentifier(spc) == spc_id:
            if spc.molecule[0].isRadical():
                top_rad_list.append((spc, final_mf))
            else:
                top_sat_list.append((spc, final_mf))

In [6]:
disp_num = 10

In [7]:
%%jinja html
top_spc_with_mf.html

Index,Species,Mole Fraction
"<a href=""http://rmg.mit.edu/database/molecule/multiplicity%202%0A1%20%20C%20u0%20p0%20c0%20%7B2%2CS%7D%20%7B3%2CS%7D%20%7B10%2CS%7D%20%7B11%2CS%7D%0A2%20%20C%20u0%20p0%20c0%20%7B1%2CS%7D%20%7B5%2CS%7D%20%7B14%2CS%7D%20%7B15%2CS%7D%0A3%20%20C%20u0%20p0%20c0%20%7B1%2CS%7D%20%7B4%2CS%7D%20%7B12%2CS%7D%20%7B13%2CS%7D%0A4%20%20C%20u0%20p0%20c0%20%7B3%2CS%7D%20%7B7%2CS%7D%20%7B18%2CS%7D%20%7B19%2CS%7D%0A5%20%20C%20u0%20p0%20c0%20%7B2%2CS%7D%20%7B6%2CS%7D%20%7B16%2CS%7D%20%7B17%2CS%7D%0A6%20%20C%20u0%20p0%20c0%20%7B5%2CS%7D%20%7B9%2CS%7D%20%7B23%2CS%7D%20%7B24%2CS%7D%0A7%20%20C%20u0%20p0%20c0%20%7B4%2CS%7D%20%7B8%2CS%7D%20%7B21%2CS%7D%20%7B22%2CS%7D%0A8%20%20C%20u0%20p0%20c0%20%7B7%2CS%7D%20%7B25%2CS%7D%20%7B26%2CS%7D%20%7B27%2CS%7D%0A9%20%20C%20u0%20p0%20c0%20%7B6%2CS%7D%20%7B20%2CD%7D%20%7B29%2CS%7D%0A10%20H%20u0%20p0%20c0%20%7B1%2CS%7D%0A11%20H%20u0%20p0%20c0%20%7B1%2CS%7D%0A12%20H%20u0%20p0%20c0%20%7B3%2CS%7D%0A13%20H%20u0%20p0%20c0%20%7B3%2CS%7D%0A14%20H%20u0%20p0%20c0%20%7B2%2CS%7D%0A15%20H%20u0%20p0%20c0%20%7B2%2CS%7D%0A16%20H%20u0%20p0%20c0%20%7B5%2CS%7D%0A17%20H%20u0%20p0%20c0%20%7B5%2CS%7D%0A18%20H%20u0%20p0%20c0%20%7B4%2CS%7D%0A19%20H%20u0%20p0%20c0%20%7B4%2CS%7D%0A20%20C%20u0%20p0%20c0%20%7B9%2CD%7D%20%7B28%2CS%7D%20%7B30%2CS%7D%0A21%20H%20u0%20p0%20c0%20%7B7%2CS%7D%0A22%20H%20u0%20p0%20c0%20%7B7%2CS%7D%0A23%20H%20u0%20p0%20c0%20%7B6%2CS%7D%0A24%20H%20u0%20p0%20c0%20%7B6%2CS%7D%0A25%20H%20u0%20p0%20c0%20%7B8%2CS%7D%0A26%20H%20u0%20p0%20c0%20%7B8%2CS%7D%0A27%20H%20u0%20p0%20c0%20%7B8%2CS%7D%0A28%20C%20u1%20p0%20c0%20%7B20%2CS%7D%20%7B31%2CS%7D%20%7B32%2CS%7D%0A29%20H%20u0%20p0%20c0%20%7B9%2CS%7D%0A30%20H%20u0%20p0%20c0%20%7B20%2CS%7D%0A31%20H%20u0%20p0%20c0%20%7B28%2CS%7D%0A32%20H%20u0%20p0%20c0%20%7B28%2CS%7D%0A"" title=""Search on RMG website"" class=""searchlink"">30.","<a href=""http://rmg.mit.edu/database/molecule/multiplicity%202%0A1%20%20C%20u0%20p0%20c0%20%7B2%2CS%7D%20%7B3%2CS%7D%20%7B10%2CS%7D%20%7B11%2CS%7D%0A2%20%20C%20u0%20p0%20c0%20%7B1%2CS%7D%20%7B5%2CS%7D%20%7B14%2CS%7D%20%7B15%2CS%7D%0A3%20%20C%20u0%20p0%20c0%20%7B1%2CS%7D%20%7B4%2CS%7D%20%7B12%2CS%7D%20%7B13%2CS%7D%0A4%20%20C%20u0%20p0%20c0%20%7B3%2CS%7D%20%7B7%2CS%7D%20%7B18%2CS%7D%20%7B19%2CS%7D%0A5%20%20C%20u0%20p0%20c0%20%7B2%2CS%7D%20%7B6%2CS%7D%20%7B16%2CS%7D%20%7B17%2CS%7D%0A6%20%20C%20u0%20p0%20c0%20%7B5%2CS%7D%20%7B9%2CS%7D%20%7B23%2CS%7D%20%7B24%2CS%7D%0A7%20%20C%20u0%20p0%20c0%20%7B4%2CS%7D%20%7B8%2CS%7D%20%7B21%2CS%7D%20%7B22%2CS%7D%0A8%20%20C%20u0%20p0%20c0%20%7B7%2CS%7D%20%7B25%2CS%7D%20%7B26%2CS%7D%20%7B27%2CS%7D%0A9%20%20C%20u0%20p0%20c0%20%7B6%2CS%7D%20%7B20%2CD%7D%20%7B29%2CS%7D%0A10%20H%20u0%20p0%20c0%20%7B1%2CS%7D%0A11%20H%20u0%20p0%20c0%20%7B1%2CS%7D%0A12%20H%20u0%20p0%20c0%20%7B3%2CS%7D%0A13%20H%20u0%20p0%20c0%20%7B3%2CS%7D%0A14%20H%20u0%20p0%20c0%20%7B2%2CS%7D%0A15%20H%20u0%20p0%20c0%20%7B2%2CS%7D%0A16%20H%20u0%20p0%20c0%20%7B5%2CS%7D%0A17%20H%20u0%20p0%20c0%20%7B5%2CS%7D%0A18%20H%20u0%20p0%20c0%20%7B4%2CS%7D%0A19%20H%20u0%20p0%20c0%20%7B4%2CS%7D%0A20%20C%20u0%20p0%20c0%20%7B9%2CD%7D%20%7B28%2CS%7D%20%7B30%2CS%7D%0A21%20H%20u0%20p0%20c0%20%7B7%2CS%7D%0A22%20H%20u0%20p0%20c0%20%7B7%2CS%7D%0A23%20H%20u0%20p0%20c0%20%7B6%2CS%7D%0A24%20H%20u0%20p0%20c0%20%7B6%2CS%7D%0A25%20H%20u0%20p0%20c0%20%7B8%2CS%7D%0A26%20H%20u0%20p0%20c0%20%7B8%2CS%7D%0A27%20H%20u0%20p0%20c0%20%7B8%2CS%7D%0A28%20C%20u1%20p0%20c0%20%7B20%2CS%7D%20%7B31%2CS%7D%20%7B32%2CS%7D%0A29%20H%20u0%20p0%20c0%20%7B9%2CS%7D%0A30%20H%20u0%20p0%20c0%20%7B20%2CS%7D%0A31%20H%20u0%20p0%20c0%20%7B28%2CS%7D%0A32%20H%20u0%20p0%20c0%20%7B28%2CS%7D%0A"">",1.917909e-09
"<a href=""http://rmg.mit.edu/database/molecule/multiplicity%202%0A1%20%20C%20u0%20p0%20c0%20%7B2%2CS%7D%20%7B4%2CS%7D%20%7B18%2CS%7D%20%7B19%2CS%7D%0A2%20%20C%20u0%20p0%20c0%20%7B1%2CS%7D%20%7B3%2CS%7D%20%7B16%2CS%7D%20%7B17%2CS%7D%0A3%20%20C%20u0%20p0%20c0%20%7B2%2CS%7D%20%7B6%2CS%7D%20%7B22%2CS%7D%20%7B23%2CS%7D%0A4%20%20C%20u0%20p0%20c0%20%7B1%2CS%7D%20%7B5%2CS%7D%20%7B20%2CS%7D%20%7B21%2CS